# import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import keras , os , tqdm , cv2
from keras.models import Sequential
from keras.layers import Dense , Conv2D , MaxPooling2D , BatchNormalization , Dropout , Flatten
from keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load Data

In [ ]:
trainDir = '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
testDir = '/kaggle/input/asl-alphabet/asl_alphabet_test/asl_alphabet_test'

In [ ]:
def loadTrsinData (trainDir , imageWidth , imageHight) :
  classes = os.listdir(trainDir)
  imagesList = []
  labels = []
  for clas in tqdm.tqdm(classes) :
    classesPath = os.path.join(trainDir,clas)
    for image in os.listdir(classesPath) :
      imgPath = os.path.join(trainDir,clas,image)
      img = cv2.imread(imgPath)
      img = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)
      img = cv2.resize(img , (imageWidth , imageHight))
      imagesList.append(img)
      labels.append(clas)

  return imagesList , labels

# Explore Data

In [ ]:
def displaySampleOfData (trainDir , imageWidth , imageHight) :
  plt.figure(figsize=(10,15))
  classes = os.listdir(trainDir)
  for i,clas in tqdm.tqdm(enumerate(classes)):
    plt.subplot(6,5,i+1)
    classesPath = os.path.join(trainDir,clas)
    image = os.listdir(classesPath)[0]
    image = os.path.join(trainDir,clas,image)
    img = cv2.imread(image)
    img = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img , (imageWidth , imageHight))
    plt.title(clas)
    plt.imshow(img , cmap='gray')
  plt.show()


In [ ]:
displaySampleOfData(trainDir ,60 ,60 )

In [ ]:
# load data after explore the images and study it's case
X , y = loadTrsinData(trainDir , 60 , 60)

In [ ]:
testImages = []
testLabels = []
for img in tqdm.tqdm(os.listdir(testDir)):
  trainImagePath = os.path.join(testDir , img)
  image = cv2.imread(trainImagePath)
  image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
  image = cv2.resize(image,(60,60))
  testImages.append(image)
  testLabels.append(img)

# Data preprocessing

## train

In [ ]:
# shuffles x and y to make better training
XShuffled , yShuffled = shuffle(X,y,random_state=42)

In [ ]:
# convert list to np array
xtrain = np.array(XShuffled)
ytrain = np.array(yShuffled)

In [ ]:
# shape of xtrain
xtrain.shape

In [ ]:
# Scale the train data
xtrain = xtrain.astype('float32') / 255.0

In [ ]:
# Reshape xtrain to fit conv network
xtrainReshaped = xtrain.reshape( (87000, 60, 60 , 1) )

In [ ]:
xtrainReshaped.shape

In [ ]:
# Create list of classes and dic to convert y labels to numbers
cats = [i for i in os.listdir(trainDir)]
categories = {}
for i,c in enumerate(cats) :
  categories[c] = i

In [ ]:
# convert labels in ytrain to numbers
for i in range (len(ytrain)) :
  ytrain[i] = categories[ytrain[i]]

ytrain

In [ ]:
# Convert ytrain from numpy array to categoricl formate to fit in the training
ytrain = to_categorical(ytrain)

## test

In [ ]:
testImages = np.array(testImages)
testLabels = np.array(testLabels)

In [ ]:
testImages = testImages.astype('float32') / 255.0

In [ ]:
testImages = testImages.reshape( (-1, 60, 60 , 1) )

In [ ]:
for i in range(len(testLabels)) :
  testLabels[i] = testLabels[i].split('_')[0]

testLabels

In [ ]:
testDic = {}
for i,c in enumerate(testLabels):
  testDic[c]=i

In [ ]:
for i in range( len(testLabels) ):
  testLabels[i] = testDic[testLabels[i]]

In [ ]:
testLabels

In [ ]:
testLabels = to_categorical(testLabels , num_classes=29)

In [ ]:
testImages = np.array(testImages, dtype=np.float32)
testLabels = np.array(testLabels, dtype=np.int32)

# Data Modeling

In [ ]:
Model = Sequential ([
    Conv2D(128 , (3,3) , activation='relu' , input_shape =(60,60,1)),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Conv2D(64 , (3,3) , activation='relu' ),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Conv2D(32 , (3,3) , activation='relu' ),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(256 , activation='relu'),
    Dropout(0.2),
    Dense(128 , activation='relu'),
    Dropout(0.15),
    Dense(64 , activation='relu'),
    Dense(len(cats) , activation='softmax'),
])
Model.summary()

In [ ]:
Model.compile(optimizer='adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])

In [ ]:
# Add callbacks for better training
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint("best_custom_cnn.keras", save_best_only=True)
]

In [ ]:
Model.fit(xtrainReshaped , ytrain  , validation_split=0.2 , epochs = 10, callbacks=callbacks)

In [ ]:
plt.plot(Model.history.history['accuracy'], label='Train Accuracy')
plt.plot(Model.history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Save the custom CNN model
Model.save('custom_cnn_asl_model.keras')
print("Custom CNN model saved as 'custom_cnn_asl_model.keras'")

# Also save in .h5 format for compatibility
Model.save('custom_cnn_asl_model.h5')
print("Custom CNN model also saved as 'custom_cnn_asl_model.h5'")

# Save the label mapping for the custom model
import json
label_mapping = {v: k for k, v in categories.items()}  # reverse mapping (index -> class name)
with open('custom_cnn_labels.json', 'w') as f:
    json.dump(label_mapping, f)
print("Label mapping saved as 'custom_cnn_labels.json'")

VGG-16

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

print("TF version:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))


In [ ]:
trainDir = '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
testDir  = '/kaggle/input/asl-alphabet/asl_alphabet_test/asl_alphabet_test'

# Vérifie le dossier
assert os.path.isdir(trainDir), f"trainDir introuvable: {trainDir}"

# Récupère et trie les classes (un dossier = une classe)
classes = sorted([d for d in os.listdir(trainDir) if os.path.isdir(os.path.join(trainDir, d))])
num_classes = len(classes)
print("Nombre de classes détectées :", num_classes)
print(classes)


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32   # si OOM, baisser à 16 ou 8
SEED = 42

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    trainDir,
    target_size=IMG_SIZE,
    color_mode='rgb',            # important : RGB pour VGG16
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=SEED
)

val_gen = datagen.flow_from_directory(
    trainDir,
    target_size=IMG_SIZE,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=SEED
)

print("Mapping classes -> indices :", train_gen.class_indices)


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Geler toutes les couches pour commencer (feature extractor)
for layer in base_model.layers:
    layer.trainable = False

print("Nombre de couches VGG16 :", len(base_model.layers))
base_model.summary()


In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2, verbose=1),
    ModelCheckpoint("vgg16_transfer_asl_best.keras", monitor='val_accuracy', save_best_only=True, verbose=1)
]

In [ ]:
EPOCHS = 10   # ou 8-12 selon le temps

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)


In [ ]:
# Save the final VGG16 transfer learning model
model.save('vgg16_transfer_asl_final.keras')
print("VGG16 transfer learning model saved as 'vgg16_transfer_asl_final.keras'")

# Also save in .h5 format for compatibility
model.save('vgg16_transfer_asl_final.h5')
print("VGG16 transfer learning model also saved as 'vgg16_transfer_asl_final.h5'")

# Save the label mapping for VGG16 model (from ImageDataGenerator)
import json
vgg16_label_mapping = {v: k for k, v in train_gen.class_indices.items()}  # reverse mapping (index -> class name)
with open('vgg16_transfer_labels.json', 'w') as f:
    json.dump(vgg16_label_mapping, f)
print("VGG16 label mapping saved as 'vgg16_transfer_labels.json'")

# Print training history summary
print(f"\nTraining completed!")
print(f"Best validation accuracy: {max(history.history['val_accuracy']):.4f}")
print(f"Final training accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")

In [ ]:
# Model Summary and Saved Files
print("="*60)
print("MODEL TRAINING COMPLETED - SUMMARY")
print("="*60)
print("\n SAVED MODEL FILES:")
print("1. Custom CNN Model:")
print("   - custom_cnn_asl_model.keras")
print("   - custom_cnn_asl_model.h5")
print("   - custom_cnn_labels.json")
print("\n2. VGG16 Transfer Learning Model:")
print("   - vgg16_transfer_asl_final.keras (final model)")
print("   - vgg16_transfer_asl_final.h5 (final model)")
print("   - vgg16_transfer_asl_best.keras (best checkpoint)")
print("   - vgg16_transfer_labels.json")
print("\n MODEL SPECIFICATIONS:")
print("1. Custom CNN:")
print("   - Input: 60x60 grayscale images")
print("   - Architecture: Custom 3-layer CNN")
print("   - Classes: 29 ASL alphabet signs")
print("\n2. VGG16 Transfer Learning:")
print("   - Input: 224x224 RGB images")
print("   - Architecture: VGG16 + custom classification head")
print("   - Classes: 29 ASL alphabet signs")
print("\n USAGE RECOMMENDATION:")
print("- Use VGG16 model for better accuracy (transfer learning)")
print("- Use Custom CNN for faster inference (smaller model)")
print("- Both models ready for frontend integration!")
print("="*60)